In [11]:
import requests

OLLAMA_URL = "http://127.0.0.1:11434"
r = requests.get(OLLAMA_URL, timeout=5)
print("status:", r.status_code)
print("headers:", dict(r.headers))
print("text:", r.text[:300])

status: 200
headers: {'Content-Type': 'text/plain; charset=utf-8', 'Date': 'Wed, 07 Jan 2026 06:28:55 GMT', 'Content-Length': '17'}
text: Ollama is running


In [12]:
import requests, json

OLLAMA_URL = "http://127.0.0.1:11434"
r = requests.get(f"{OLLAMA_URL}/api/tags", timeout=10)
print("status:", r.status_code)
print(r.text[:300])


status: 200
{"models":[{"name":"smollm2:1.7b","model":"smollm2:1.7b","modified_at":"2026-01-07T15:22:51.685801521+09:00","size":1820428533,"digest":"cef4a1e09247f018ca0c482ad4c2ce1474aba5e87f245dacf97f07948d05d8b4","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_siz


In [5]:
import requests, json

OLLAMA_URL = "http://127.0.0.1:11434"

tags = requests.get(f"{OLLAMA_URL}/api/tags", timeout=10).json()
print("models:", [m["name"] for m in tags.get("models", [])][:20])


models: ['smollm2:1.7b']


In [14]:
import json
import requests
from typing import Any, Dict

OLLAMA_URL = "http://127.0.0.1:11434"
MODEL_NAME = "smollm2:1.7b"

def extract_first_json_object(text: str) -> str:
    start = text.find("{")
    if start == -1:
        raise ValueError("No '{' found in output")

    depth = 0
    in_str = False
    esc = False

    for i in range(start, len(text)):
        ch = text[i]

        if in_str:
            if esc:
                esc = False
            elif ch == "\\":
                esc = True
            elif ch == '"':
                in_str = False
            continue

        if ch == '"':
            in_str = True
        elif ch == "{":
            depth += 1
        elif ch == "}":
            depth -= 1
            if depth == 0:
                return text[start:i+1]

    raise ValueError("Unbalanced braces; cannot extract JSON.")


In [15]:
def ollama_generate(prompt: str, temperature: float = 0.0, num_ctx: int = 4096) -> str:
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": temperature,
            "num_ctx": num_ctx,
        },
    }
    r = requests.post(f"{OLLAMA_URL}/api/generate", json=payload, timeout=120)
    r.raise_for_status()
    return r.json()["response"]

def ollama_json(user_prompt: str, schema_hint: str, temperature: float = 0.0) -> Dict[str, Any]:
    system = (
        "You are a JSON-only assistant. "
        "Return ONLY one valid JSON object. "
        "No markdown, no explanations, no extra text."
    )

    prompt = f"""{system}

Return ONLY JSON matching this schema:
{schema_hint}

REQUEST:
{user_prompt}
""".strip()

    raw = ollama_generate(prompt, temperature=temperature).strip()

    try:
        return json.loads(extract_first_json_object(raw))
    except Exception:
        # 1회 재시도: JSON만 고치게 강제
        repair = f"""{system}

Fix the following output to be ONLY valid JSON matching schema:
{schema_hint}

RAW OUTPUT:
{raw}
""".strip()
        raw2 = ollama_generate(repair, temperature=0.0).strip()
        return json.loads(extract_first_json_object(raw2))


In [16]:
schema_hint = """
{
  "category": "one of [finance, legal, tech, medical, general, other]",
  "summary": "string (<= 120 chars)",
  "keywords": "array of 3~8 strings",
  "confidence": "number (0.0~1.0)"
}
""".strip()

doc = "삼성전자는 차세대 반도체 공정 로드맵을 발표했으며, 2나노 공정 확대 계획을 밝혔다..."

user_prompt = f"""
DOCUMENT (Korean):
{doc}

TASK:
Classify category, summarize <=120 chars, give 3~8 keywords, and confidence.
""".strip()

result = ollama_json(user_prompt, schema_hint, temperature=0.0)
print(json.dumps(result, ensure_ascii=False, indent=2))


{
  "category": "tech",
  "summary": "Samsung Electronics unveils a new battery technology for its smartphones.",
  "keywords": [
    "battery technology",
    "smartphones"
  ],
  "confidence": 0.95
}


In [17]:
print(json.dumps(result, ensure_ascii=False, indent=2))


{
  "category": "tech",
  "summary": "Samsung Electronics unveils a new battery technology for its smartphones.",
  "keywords": [
    "battery technology",
    "smartphones"
  ],
  "confidence": 0.95
}
